### Loading

The archive provided is a .rar file with a .arff inside.  Before anything, we must load this information and transform it to a Pandas DataFrame object.

In [70]:
import pandas as pd
import numpy as np
import os

root_dir = os.getcwd()

In [71]:
# NOTE: ONLY RUN THIS ONCE

#import patoolib

#   decompress Chronic_Kidney_Disease.rar
#patoolib.extract_archive("Chronic_Kidney_Disease.rar", outdir=root_dir)

#   move Chronic_Kidney_Disease.arff to root directory
#os.rename(os.path.join(root_dir, "Chronic_Kidney_Disease", "chronic_kidney_disease_full.arff"), os.path.join(root_dir, "Chronic_Kidney_Disease.arff"))

#   convert .arff to .csv with arffToCsv.py
#os.system("python arffToCsv.py")


##### NOTE:

There are stray commas on lines 71, 74, 371.  Likely input errors.
New data should have the correct number of columns.  Fix manually in Excel or a text editor if necessary.

In [72]:
# load Chronic_Kidney_Disease.csv to pandas dataframe
df = pd.read_csv("Chronic_Kidney_Disease.csv")

# count number of '?' in each column
print(df.isin(['?']).sum())

'age'        9
'bp'        12
'sg'        47
'al'        46
'su'        49
'rbc'      152
'pc'        65
'pcc'        4
'ba'         4
'bgr'       44
'bu'        19
'sc'        17
'sod'       87
'pot'       88
'hemo'      52
'pcv'       70
'wbcc'     105
'rbcc'     130
'htn'        2
'dm'         2
'cad'        2
'appet'      1
'pe'         1
'ane'        1
'class'      0
dtype: int64


### Munging

Now that we have loaded the data, we must clean the data for null values and make sure all of the data types make sense.  It may be helpful to scale some of the numerical values.

In [73]:
# display first 5 rows of dataframe
df.head()

,'age','bp','sg','al','su','rbc','pc','pcc','ba','bgr',...,'pcv','wbcc','rbcc','htn','dm','cad','appet','pe','ane','class'
0,48,80,1.020,1,0,?,normal,notpresent,notpresent,121,...,44,7800,5.2,yes,yes,no,good,no,no,ckd
1,7,50,1.020,4,0,?,normal,notpresent,notpresent,?,...,38,6000,?,no,no,no,good,no,no,ckd
2,62,80,1.010,2,3,normal,normal,notpresent,notpresent,423,...,31,7500,?,no,yes,no,poor,no,yes,ckd
3,48,70,1.005,4,0,normal,abnormal,present,notpresent,117,...,32,6700,3.9,yes,no,no,poor,yes,yes,ckd
4,51,80,1.010,2,0,normal,normal,notpresent,notpresent,106,...,35,7300,4.6,no,no,no,good,no,no,ckd


Here we can see that there are three data columns which are registered '?' for over a quarter of the data set.  If we remove all observations with null values at this stage, we will lose a huge portion of available information.  A later study may include a regression model based on the other columns to fill the missing data, but for practical constraints we have to delete the columns with the most missing information.

Also... these column names are ridiculous.

In [74]:
# list of column names
names = ['age', 'blood_pressure', 'specific_gravity', 'albumin', 'sugar', 'red_blood_cells',
         'pus_cell', 'pus_cell_clumps', 'bacteria', 'blood_glucose_random', 'blood_urea',
         'serum_creatinine', 'sodium', 'potassium', 'hemoglobin', 'packed_cell_volume',
         'white_blood_cell_count', 'red_blood_cell_count', 'hypertension', 'diabetes_mellitus',
         'coronary_artery_disease', 'appetite', 'pedal_edema', 'anemia', 'class']

# rename all the columns
df.columns = names

In [75]:
remove = ['red_blood_cells', 'red_blood_cell_count', 'white_blood_cell_count']

# remove columns with too many missing values
df = df.drop(remove, axis=1)
df.describe()

,age,blood_pressure,specific_gravity,albumin,sugar,pus_cell,pus_cell_clumps,bacteria,blood_glucose_random,blood_urea,...,potassium,hemoglobin,packed_cell_volume,hypertension,diabetes_mellitus,coronary_artery_disease,appetite,pedal_edema,anemia,class
count,400,400,400,400,400,400,400,400,400,400,...,400,400,400,400,400,400,400,400,400,400
unique,77,11,6,7,7,3,3,3,147,119,...,43,119,45,3,6,4,3,3,3,3
top,60,80,1.020,0,0,normal,notpresent,notpresent,?,?,...,?,?,?,no,no,no,good,no,no,ckd
freq,19,116,106,199,290,259,354,374,44,19,...,88,52,70,251,258,362,317,323,339,248


In [76]:
# show all unique values in each column
for col in df.columns:
    print(col, df[col].unique())

age ['48' '7' '62' '51' '60' '68' '24' '52' '53' '50' '63' '40' '47' '61' '21'
 '42' '75' '69' '?' '73' '70' '65' '76' '72' '82' '46' '45' '35' '54' '11'
 '59' '67' '15' '55' '44' '26' '64' '56' '5' '74' '38' '58' '71' '34' '17'
 '12' '43' '41' '57' '8' '39' '66' '81' '14' '27' '83' '30' '4' '3' '6'
 '32' '80' '49' '90' '78' '19' '2' '33' '36' '37' '23' '25' '20' '29' '28'
 '22' '79']
blood_pressure ['80' '50' '70' '90' '?' '100' '60' '110' '140' '180' '120']
specific_gravity ['1.020' '1.010' '1.005' '1.015' '?' '1.025']
albumin ['1' '4' '2' '3' '0' '?' '5']
sugar ['0' '3' '4' '1' '?' '2' '5']
pus_cell ['normal' 'abnormal' '?']
pus_cell_clumps ['notpresent' 'present' '?']
bacteria ['notpresent' 'present' '?']
blood_glucose_random ['121' '?' '423' '117' '106' '74' '100' '410' '138' '70' '490' '380' '208'
 '98' '157' '76' '99' '114' '263' '173' '95' '108' '156' '264' '123' '93'
 '107' '159' '140' '171' '270' '92' '137' '204' '79' '207' '124' '144'
 '91' '162' '246' '253' '141' '182' '86'

Some things that stand out:

 * The independent variable has an extra value 'ckd\t'.  The other two values are (0,1). 4 more columns contain this error.
 * Almost every dependent variable contains null values
 * Everything is represented by a string
 * Sometimes it's okay to have unknown values

In [77]:
# explore 'class' column
df['class'].value_counts()

ckd       248
notckd    150
ckd\t       2
Name: class, dtype: int64

Rows with the weird value are input errors.  Other columns are also contaminated with \t.
Other binary factors get converted to binary.

In [78]:
# columns with \t in values are packed_cell_volume, diabetes_mellitus, coronary_artery_disease, and class
# remove \t from values
df['packed_cell_volume'] = df['packed_cell_volume'].str.replace('\t', '')
df['diabetes_mellitus'] = df['diabetes_mellitus'].str.replace('\t', '')
df['coronary_artery_disease'] = df['coronary_artery_disease'].str.replace('\t', '')
df['class'] = df['class'].str.replace('\t', '')


In [79]:
df

,age,blood_pressure,specific_gravity,albumin,sugar,pus_cell,pus_cell_clumps,bacteria,blood_glucose_random,blood_urea,...,potassium,hemoglobin,packed_cell_volume,hypertension,diabetes_mellitus,coronary_artery_disease,appetite,pedal_edema,anemia,class
0,48,80,1.020,1,0,normal,notpresent,notpresent,121,36,...,?,15.4,44,yes,yes,no,good,no,no,ckd
1,7,50,1.020,4,0,normal,notpresent,notpresent,?,18,...,?,11.3,38,no,no,no,good,no,no,ckd
2,62,80,1.010,2,3,normal,notpresent,notpresent,423,53,...,?,9.6,31,no,yes,no,poor,no,yes,ckd
3,48,70,1.005,4,0,abnormal,present,notpresent,117,56,...,2.5,11.2,32,yes,no,no,poor,yes,yes,ckd
4,51,80,1.010,2,0,normal,notpresent,notpresent,106,26,...,?,11.6,35,no,no,no,good,no,no,ckd
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
395,55,80,1.020,0,0,normal,notpresent,notpresent,140,49,...,4.9,15.7,47,no,no,no,good,no,no,notckd
396,42,70,1.025,0,0,normal,notpresent,notpresent,75,31,...,3.5,16.5,54,no,no,no,good,no,no,notckd
397,12,80,1.020,0,0,normal,notpresent,notpresent,100,26,...,4.4,15.8,49,no,no,no,good,no,no,notckd
398,17,60,1.025,0,0,normal,notpresent,notpresent,114,50,...,4.9,14.2,51,no,no,no,good,no,no,notckd


In [80]:
# replace ? with NaN
df = df.replace('?', np.nan)

# drop rows with NaN values
df = df.dropna()
df

,age,blood_pressure,specific_gravity,albumin,sugar,pus_cell,pus_cell_clumps,bacteria,blood_glucose_random,blood_urea,...,potassium,hemoglobin,packed_cell_volume,hypertension,diabetes_mellitus,coronary_artery_disease,appetite,pedal_edema,anemia,class
3,48,70,1.005,4,0,abnormal,present,notpresent,117,56,...,2.5,11.2,32,yes,no,no,poor,yes,yes,ckd
6,68,70,1.010,0,0,normal,notpresent,notpresent,100,54,...,4.0,12.4,36,no,no,no,good,no,no,ckd
9,53,90,1.020,2,0,abnormal,present,notpresent,70,107,...,3.7,9.5,29,yes,yes,no,poor,no,yes,ckd
11,63,70,1.010,3,0,abnormal,present,notpresent,380,60,...,4.2,10.8,32,yes,yes,no,poor,yes,no,ckd
12,68,70,1.015,3,1,normal,present,notpresent,208,72,...,5.8,9.7,28,yes,yes,yes,poor,yes,no,ckd
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
395,55,80,1.020,0,0,normal,notpresent,notpresent,140,49,...,4.9,15.7,47,no,no,no,good,no,no,notckd
396,42,70,1.025,0,0,normal,notpresent,notpresent,75,31,...,3.5,16.5,54,no,no,no,good,no,no,notckd
397,12,80,1.020,0,0,normal,notpresent,notpresent,100,26,...,4.4,15.8,49,no,no,no,good,no,no,notckd
398,17,60,1.025,0,0,normal,notpresent,notpresent,114,50,...,4.9,14.2,51,no,no,no,good,no,no,notckd


In [81]:
# convert normal to 0 in every column
for col in df.columns:
    df[col] = df[col].replace('normal', 0)

# convert abnormal to 1 in every column
for col in df.columns:
    df[col] = df[col].replace('abnormal', 1)

# convert present to 1 in every column
for col in df.columns:
    df[col] = df[col].replace('present', 1)

# convert notpresent to 0 in every column
for col in df.columns:
    df[col] = df[col].replace('notpresent', 0)

# convert yes to 1 in every column
for col in df.columns:
    df[col] = df[col].replace('yes', 1)

# convert no to 0 in every column
for col in df.columns:
    df[col] = df[col].replace('no', 0)
# convert good to 1
for col in df.columns:
    df[col] = df[col].replace('good', 1)

# convert poor to 0
for col in df.columns:
    df[col] = df[col].replace('poor', 0)

# convert ckd to 1
df['class'] = df['class'].replace('ckd', 1)

# convert notckd to 0
df['class'] = df['class'].replace('notckd', 0)


In [82]:
df

,age,blood_pressure,specific_gravity,albumin,sugar,pus_cell,pus_cell_clumps,bacteria,blood_glucose_random,blood_urea,...,potassium,hemoglobin,packed_cell_volume,hypertension,diabetes_mellitus,coronary_artery_disease,appetite,pedal_edema,anemia,class
3,48,70,1.005,4,0,1,1,0,117,56,...,2.5,11.2,32,1,0,0,0,1,1,1
6,68,70,1.010,0,0,0,0,0,100,54,...,4.0,12.4,36,0,0,0,1,0,0,1
9,53,90,1.020,2,0,1,1,0,70,107,...,3.7,9.5,29,1,1,0,0,0,1,1
11,63,70,1.010,3,0,1,1,0,380,60,...,4.2,10.8,32,1,1,0,0,1,0,1
12,68,70,1.015,3,1,0,1,0,208,72,...,5.8,9.7,28,1,1,1,0,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
395,55,80,1.020,0,0,0,0,0,140,49,...,4.9,15.7,47,0,0,0,1,0,0,0
396,42,70,1.025,0,0,0,0,0,75,31,...,3.5,16.5,54,0,0,0,1,0,0,0
397,12,80,1.020,0,0,0,0,0,100,26,...,4.4,15.8,49,0,0,0,1,0,0,0
398,17,60,1.025,0,0,0,0,0,114,50,...,4.9,14.2,51,0,0,0,1,0,0,0


In [85]:
# convert all values to numeric
df = df.apply(pd.to_numeric)

Now we can scale all the values between zero and 1.

In [86]:
# scale all values to be between 0 and 1
for col in df.columns:
    df[col] = df[col] / df[col].max()

In [87]:
df

,age,blood_pressure,specific_gravity,albumin,sugar,pus_cell,pus_cell_clumps,bacteria,blood_glucose_random,blood_urea,...,potassium,hemoglobin,packed_cell_volume,hypertension,diabetes_mellitus,coronary_artery_disease,appetite,pedal_edema,anemia,class
3,0.533333,0.636364,0.980488,1.00,0.0,1.0,1.0,0.0,0.238776,0.181230,...,0.053191,0.629213,0.592593,1.0,0.0,0.0,0.0,1.0,1.0,1.0
6,0.755556,0.636364,0.985366,0.00,0.0,0.0,0.0,0.0,0.204082,0.174757,...,0.085106,0.696629,0.666667,0.0,0.0,0.0,1.0,0.0,0.0,1.0
9,0.588889,0.818182,0.995122,0.50,0.0,1.0,1.0,0.0,0.142857,0.346278,...,0.078723,0.533708,0.537037,1.0,1.0,0.0,0.0,0.0,1.0,1.0
11,0.700000,0.636364,0.985366,0.75,0.0,1.0,1.0,0.0,0.775510,0.194175,...,0.089362,0.606742,0.592593,1.0,1.0,0.0,0.0,1.0,0.0,1.0
12,0.755556,0.636364,0.990244,0.75,0.2,0.0,1.0,0.0,0.424490,0.233010,...,0.123404,0.544944,0.518519,1.0,1.0,1.0,0.0,1.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
395,0.611111,0.727273,0.995122,0.00,0.0,0.0,0.0,0.0,0.285714,0.158576,...,0.104255,0.882022,0.870370,0.0,0.0,0.0,1.0,0.0,0.0,0.0
396,0.466667,0.636364,1.000000,0.00,0.0,0.0,0.0,0.0,0.153061,0.100324,...,0.074468,0.926966,1.000000,0.0,0.0,0.0,1.0,0.0,0.0,0.0
397,0.133333,0.727273,0.995122,0.00,0.0,0.0,0.0,0.0,0.204082,0.084142,...,0.093617,0.887640,0.907407,0.0,0.0,0.0,1.0,0.0,0.0,0.0
398,0.188889,0.545455,1.000000,0.00,0.0,0.0,0.0,0.0,0.232653,0.161812,...,0.104255,0.797753,0.944444,0.0,0.0,0.0,1.0,0.0,0.0,0.0


We are left with 209 rows of clean scaled data from an initial lot of 400 filthy rows.  This is a considerable loss of information, however, the data we have is clean.

### Saving the data

The clean data should be saved as a .csv, so that it can easily be loaded with Pandas or manipulated in Excel.

In [88]:
# save dataframe to csv
df.to_csv('CKD_clean.csv', index=False)

Please proceed to "presentation.ipynb" for the rest of the project.